## Análise de Dados do COVID-19

Análise de séries temporais sobre a contaminação do COVID-19.

Dataset utilizado: [Kagle](https://www.kaggle.com/imdevskp/corona-virus-report).

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
#Importando os dados
df = pd.read_csv('covid_19_clean_complete.csv', parse_dates=['Date'])
df

Antes de iniciarmos a análise existe um fator que devemos nos atentar: 
    
* Os nomes das colunas não podem ter caracteres especiais. 

E para facilitar as análises esses nomes estarão todos em letras minúsculas.


In [ ]:
import re
def editando_colunas(col_name):
    return re.sub(r"[/| ]", "", col_name).lower()

In [ ]:
df.columns = [editando_colunas(col) for col in df.columns]

df

Antes de iniciarmos as análises devemos ter informações sobre o Data Set analisado:

* Tipos de variáveis.

* Número de dados do Data Set.

* Quantidade de dados nulos.

* Quantidade de dados únicos.

In [ ]:
#Informações sobre o DataSet analisado
covid = pd.DataFrame({'colunas' : df.columns,
                    'tipo': df.dtypes,
                    'nulos' : df.isna().sum(),
                    'quantidade' : df.shape[0],
                    'unicos': df.nunique()})
covid['percentual'] = round(covid['nulos'] / covid['quantidade'],2)

covid

Observando as colunas percebemos que apenas a coluna Province/State possui 70% dos seus valores nulos.

In [ ]:
#Data Inicial
print(min(df['date']))
#Data final 
print(max(df['date']))

O Data Set engloba os casos de covid a partir do dia 22/01/2020 até o dia 19/05/2020.


### Análises dos dados

Vamos analisar os dados do Brasil:

**Primeiro passo:** investigar as variáveis que temos a nossa disposição. 

* Temos as informações disponíveis por Estado?
* Quais variáveis temos disponíveis e podemos trabalhar com essa base de dados?

In [ ]:
df.loc[df.countryregion == 'Brazil']


No Brasil não temos a informação de casos de COVID-19 a nível de estado. Verificaremos algumas outras varíaveis para verificar o avanço do COVID-19 no país.

### Casos Confirmados

A primeira análise diz respeito a curva dos casos confirmados.

In [ ]:
brasil = df.loc[(df.countryregion == 'Brazil') & (df.confirmed > 0)]

In [ ]:
px.line(brasil, 'date', 'confirmed',
        labels={'date':'Data', 'confirmed':'Numero de casos confirmados'},
        title = 'Casos confirmados de COVID-19 no Brasil')

### Número de casos por dia

Para verificar o número de casos por dia devemos antes implementar uma função.

In [ ]:
#Calcula o numero de casos dia
brasil['novoscasos'] = list(map(
    lambda x: 0 if (x==0) else brasil['confirmed'].iloc[x] - brasil['confirmed'].iloc[x-1],
    np.arange(brasil.shape[0])
))

In [ ]:
px.line(brasil, x = 'date', y='novoscasos', title='Novos casos de COVID-19 por dia no Brasil',
        labels={'date':'Data', 'novoscasos':'Novos casos'})


### Mortes por COVID-19

Análise do número de mortes no Brasil pelo COVID-19.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter( x =brasil.date, y=brasil.deaths, name='Mortes', mode='lines+markers',
                         line=dict(color='red')))

fig.update_layout(title='Mortes por COVID-19 no Brasil', xaxis_title='Data', yaxis_title='Número de mortes')

fig.show()

Através desde gráfico observa-se que o número de mortos pelo COVID-19 no país segue uma curva exponencial.

### Taxa de crescimento do COVID-19


Verificaremos de forma gráfica a taxa de crescimento por dia dos casos de COVID-19 confirmados no Brasil.



In [ ]:
def taxa_crescimento_diaria(data, variable, data_inicio=None):
    if data_inicio == None:
        data_inicio = data.date.loc[data[variable] > 0].min()
    else:
        data_inicio = pd.to_datetime(data_inicio)
        
    data_fim = data.date.max()
    n = (data_fim - data_inicio).days
    
    taxas = list(map(
        lambda x: (data[variable].iloc[x] - data[variable].iloc[x-1]) / data[variable].iloc[x-1],
        range(1,n+1)
    ))
    return np.array(taxas)*100

In [ ]:
tx_dia = taxa_crescimento_diaria(brasil, 'confirmed')

In [ ]:
primeiro_dia = brasil.date.loc[brasil.confirmed > 0].min()
px.line(x=pd.date_range(primeiro_dia, brasil.date.max())[1:],
        y=tx_dia, title='Taxa de crescimento de casos de COVID-19 confirmados no Brasil',
       labels={'y':'Taxa de crescimento', 'x':'Data'})